## Libraries and Dependencies

In [4]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

## Code

Constants

In [26]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.db"
USER_ID = 'Yorudan'
TOKEN = 'BQCyZJhQ7Xnkt72JUPBG7BFVNhPT7SHICmLyVJ91VaBbaK6Y2rJ1RpZGw1sPVOlTqeh-g6mxyPi5x4UrEjCtoKzg-Xk0SAU0khnam53PwSU6DYYSfZ7ismiQKkReuj3kaOghvrbVSlpTcu7O-VjgK7wC9MfuOyxyug17muRm9aKZcYUeIq-oaZ9k3kbWOE86YaWYSw'

Validation

In [13]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # check if df is empty
    if df.empty:
        print('No songs downloaded. Finishing execution')
        return False
    
    # primary key check
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception('Primary key check violated')

    # check for nulls
    if df.isnull().values.any():
        raise Exception('Null value found')

    # check that all timestamps are yesterday's date
    # yesterday = datetime.datetime.now() - datetime.timedelta(days = 1)
    # yesterday = yesterday.replace(hour = 0, minute = 0, second = 0, microsecond = 0)

    # timestamps = df['timestamp'].tolist()
    # for timestamp in timestamps:
    #     if datetime.datetime.strptime(timestamp, '%Y-%m-%d') >= yesterday:
    #         raise Exception('At least one of the returned songs does not come from within 24hrs')

    return True

Main

In [32]:
if __name__ == '__main__':
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {token}'.format(token = TOKEN)
    }

    # convert time to unix timestamp in ms
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days = 1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # download all songs i've listened in the last 24hrs
    r = requests.get('https://api.spotify.com/v1/me/player/recently-played?after={time}'.format(time=yesterday_unix_timestamp), headers = headers)
    data = r.json()
    
    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    # extract only the relevant data from json obj
    for song in data['items']:
        song_names.append(song['track']['name'])
        artist_names.append(song['track']['album']['artists'][0]['name'])
        played_at_list.append(song['played_at'])
        timestamps.append(song['played_at'][0:10])

    # prepare a dictionary in order to turn it into a pd df
    song_dict = {
        'song_name': song_names,
        'artist_name': artist_names,
        'played_at': played_at_list,
        'timestamp': timestamps
    }

    song_df = pd.DataFrame(song_dict, columns = ['song_name', 'artist_name', 'played_at', 'timestamp'])
    display(song_df)

    # validate
    if check_if_valid_data(song_df):
        print('Data valid, proceed to load stage')
        
    # create database engine and pointer
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('my_played_tracks.db')
    cursor = conn.cursor()

    # define table
    sql_query = """
    CREATE TABLE IF NOT EXISTS my_played_tracks(
      song_name varchar(200),
      artist_name varchar(200),
      played_at varchar(200),
      timestamp varchar(200),
      CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )"""

    # execute query
    cursor.execute(sql_query)
    print('Opened database successfully')

    # load dataframe into database
    try:
      song_df.to_sql('my_played_tracks', engine, index = False, if_exists='append')
    except:
      print('Data already exists')

    # close connection
    conn.close()
    print('Closed database successfully')

,song_name,artist_name,played_at,timestamp
0,Grand Escape (feat. Toko Miura),RADWIMPS,2023-02-17T10:48:20.988Z,2023-02-17
1,Dream lantern,RADWIMPS,2023-02-17T10:42:41.799Z,2023-02-17
2,Zenzenzense - movie ver.,RADWIMPS,2023-02-17T10:39:58.539Z,2023-02-17
3,君のせい,the peggies,2023-02-17T10:35:12.154Z,2023-02-17
4,センチメートル,the peggies,2023-02-17T10:30:48.957Z,2023-02-17
5,カタオモイ,Aimer,2023-02-17T10:27:01.857Z,2023-02-17
6,Body Back (feat. Maia Wright),Gryffin,2023-02-17T10:23:15.769Z,2023-02-17
7,Always Be There,Jonas Blue,2023-02-16T07:04:22.243Z,2023-02-16
8,Perfect Strangers,Jonas Blue,2023-02-16T07:01:03.332Z,2023-02-16
9,Mama,Jonas Blue,2023-02-16T06:55:59.579Z,2023-02-16


Data valid, proceed to load stage
Opened database successfully
Closed database successfully
